In [115]:
#%matplotlib inline
import datetime
from dateutil import parser
import math
from dateutil.parser import parse
from geopandas import GeoDataFrame
import pandas as pd
import numpy as np
from shapely.geometry import Point
import json
import ast
import sys
import pickle
from __future__ import print_function
import os
import time

os.chdir("/Users/sergiocamelo/Dropbox/Sergio-Joann/")

sys.path.insert(0, 'Code/VRPEngine/pyCode')
sys.path.insert(0, 'Code/VRPEngine/C++Engine')
sys.path.insert(0, 'Code/VRPEngine/pyCode/tsp')
f = open('StandardizedData/report.txt', 'w+')

import solver as solver
import distances as distances
import VRPClass

In [190]:
data_folder = '/Users/sergiocamelo/Dropbox/Sergio-Joann/StandardizedData/'
results_folder = '/Users/sergiocamelo/Dropbox/Sergio-Joann/Results/'


In [189]:
# Create timestamped folder
ts = time.time()
st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
results_folder_ts = results_folder + st + "/" 
os.mkdir(results_folder_ts)

'/Users/sergiocamelo/Dropbox/Sergio-Joann/Results/2018-09-25 21:36:41/'

In [ ]:
# Parameters for the run
season = "high"
period = 3*28 # Use three month period
days_analysis = 14

In [ ]:
# Add metadata file
metadata = {
    ""
}
with open( results_folder_ts+"json_metadata.json", "wb" ) as fp:
    json.dump(metadata, fp)

In [117]:
season_volume = season + '_volume'

In [118]:
# Load datasets
dim_farmers = pd.read_csv(data_folder+'dim_farmers.csv')
dim_middlemen = pd.read_csv(data_folder+'dim_middlemen.csv')
dim_mills = pd.read_csv(data_folder+'dim_mills.csv')
harvest_frequency_mapping = pd.read_csv(data_folder+'harvest_frequency_mapping.csv')
print("Number of plantations: %d" % (len(dim_farmers)),file=f)
print("Number of middlemen: %d" % (len(dim_middlemen)),file=f)

In [119]:
# Create a unique identifier for the farmer plot
dim_farmers['plot_id'] = [dim_farmers['farmer_id'][i] + '-'+str(dim_farmers['plot_number'][i]) for i in range(len(dim_farmers))]

In [149]:
# Create a dictionary of the farmers to cluster
farmer_to_cluster = dict(zip(dim_farmers['plot_id'],dim_farmers['cluster_id']))

In [120]:
# Calculate middleman capacity
dim_middlemen['trucks_dict']  = dim_middlemen['trucks'].map(lambda d:ast.literal_eval(d))
dim_middlemen['capacity'] = dim_middlemen['trucks_dict'].map(lambda d:np.sum([int(t)*d[t] for t in d.keys()]))

In [121]:
# Join farmers and middlemen data
# Change names of columns
dim_farmers = dim_farmers.rename(index=str, columns={"latitude": "latitude_farmer", "longitude": "longitude_farmer"})
dim_middlemen = dim_middlemen.rename(index=str, columns={"latitude": "latitude_middleman", "longitude": "longitude_middleman"})
dim_mills = dim_mills.rename(index=str, columns={"latitude": "latitude_mill", "longitude": "longitude_mill"})

result = pd.merge(dim_farmers, dim_middlemen, on=['cluster_id'], how='inner')
print("Number of plantations with middleman: %d" % (len(result)),file=f)

In [122]:
# Check if there are any duplicates
result[result.duplicated(subset=['farmer_id','plot_number'], keep=False)].to_csv(data_folder+'data_cleaning_results/duplicates.csv')
print("Total of duplicates: %d" % (len(result[result.duplicated(subset=['farmer_id','plot_number'], keep=False)])),file=f)
print ("Duplicates saved in data_cleaning_results/duplicates.csv",file=f)

In [123]:
#Use only data with lat_lon and with productions
df_full = result[np.logical_and(pd.notnull(result['longitude_farmer']),pd.notnull(result['latitude_farmer']))].copy()
print("Number of plantations with latlon: %d" % (len(df_full)),file=f)
# Use data with productions
df_full = df_full[df_full[season+'_rate']!=0].copy()
df_full = df_full[pd.notnull(df_full[season+'_rate'])].copy()
df_full = df_full[df_full[season+'_volume']!=0].copy()
print("Number of plantations that produce during the season : %d" % (len(df_full)),file=f)

In [124]:
# Map number of days
harvest_frequency_mapping_dict = {row[0]:row[1] for i,row in harvest_frequency_mapping.iterrows()}
df_full['rate'] = df_full[season+'_rate'].map(harvest_frequency_mapping_dict)

In [125]:
# Has pickup date 
df_full = df_full[pd.notnull(df_full['date_last_sold'])].copy()
print("Number of plantations with last date: %d" % (len(df_full)),file=f)

In [126]:
# Generate days of pickup
ref_day = datetime.datetime.strptime('1/1/2000', "%m/%d/%Y")
days = np.array([(parse(v)-ref_day).days for v in df_full['date_last_sold'].values])
df_full['day_mod'] = days%period
def calculate_pickup_days(row):
    d = row['day_mod']
    freq = row[season+'_rate']
    l = []
    for i in range(int(period/freq)):
        l.append((d + i * freq)%period)
    return l
df_full['pickup_days'] = df_full.apply(calculate_pickup_days, axis=1)

In [127]:
# Explote data
clusters = np.unique(df_full['cluster_id'])
df_exploted = pd.merge(df_full,df_full.pickup_days.apply(pd.Series).stack().reset_index(level=1, drop=True).to_frame('pickup'),left_index=True, right_index=True)
for c in clusters:
    print("Number of plantations of cluster %d: %d" % (c,len(df_full[df_full['cluster_id'] == c].copy())),file=f)
df_clusters = df_exploted[(np.array([c in clusters for c in df_exploted['cluster_id']])) & (df_exploted['pickup'] < days_analysis)].copy()

In [128]:
# Print middlemen data in the results folder
dim_middlemen[[dim_middlemen['cluster_id'][i] in clusters for i in range(len(dim_middlemen))]][['cluster_id','trucks','mills','capacity']].to_csv(results_folder+"tables_for_report/middlemen_data.csv", index=False)

In [129]:
# Calculate total capacity
dict_comparisons={}
for c in clusters:
    dict_comparisons[c] = {}
    dict_comparisons[c]['capacity'] = dim_middlemen[dim_middlemen.cluster_id == c]['capacity'].iloc[0]

In [130]:
# Round producing quantities to the decimal up
df_clusters[season + '_volume'] = np.ceil(df_clusters[season + '_volume'] * 10)/10

In [131]:
# Number of plantations picked up each day and quantities picked up each day
# Calculate the number of days
agg_quant = df_clusters.groupby(['cluster_id','pickup']).agg({'farmer_id':'count', season+'_volume': 'sum'})
agg_quant['overload']=agg_quant[season+'_volume']-agg_quant.apply(lambda r:dict_comparisons[r.name[0]]['capacity'],1)
outliers = (agg_quant[agg_quant['overload']>0])
print(outliers)

                   farmer_id  high_volume  overload
cluster_id pickup                                  
35         9.0             6         17.8       6.8
           10.0            6         22.8      11.8
108        4.0            10         18.3       0.3
591        9.0             6         11.7       2.7


In [132]:
# Create a report of inconsistent data
report = []
for index,row in outliers.iterrows():
    report.append({'cluster':index[0],
                   'farmers':row['farmer_id'],
                         season+'_volume':row[season+'_volume'],
                         'capacity':row[season+'_volume']-row['overload'],
                        'farmer_id-plot':[r['farmer_id']+'-'+str(r['plot_number']) for j,r in df_clusters.iterrows() if (r['cluster_id']==index[0] and r['pickup']==index[1])]})
print("Found %d trucks carrying more than their capacity" % len(outliers),file=f)
pd.DataFrame(report).to_csv(data_folder+'data_cleaning_results/overcapacity.csv')
print("Report saved in data_cleaning_results/overcapacity.csv",file=f)

In [133]:
df_clusters_c = df_clusters.copy()
moved_farmers = 0
# Modify inconsistent data, changing the pickup days randomly
for index,row in outliers.iterrows():
    success = False
    df_cluster_outlier = df_clusters_c[df_clusters_c['cluster_id']==index[0]]
    df_cluster_pickup_outlier = df_clusters_c[(df_clusters_c['cluster_id']==index[0])&(df_clusters_c['pickup']==index[1])]
    cluster_capacity = dict_comparisons[index[0]]['capacity']
    overload = np.sum(df_cluster_pickup_outlier[season+'_volume']) - cluster_capacity
    # Extract rows to move
    for index_to_replace, row_to_replace in df_cluster_pickup_outlier.iterrows():
        row_capacity = row_to_replace[season+'_volume']
        # Calculate the overloads of all pickup days
        df_cluster_outlier = df_clusters_c[df_clusters_c['cluster_id']==index[0]]
        agg_quant = df_cluster_outlier.groupby(['pickup']).agg({'farmer_id':'count', season+'_volume': 'sum'})
        agg_quant['overload']=agg_quant[season+'_volume']-cluster_capacity
        for index_to_replace_for,row_to_replace_for in agg_quant.iterrows():
            if (row_to_replace_for['overload'] + row_capacity) <= 0:
                df_clusters_c.loc[index_to_replace,'pickup'] = index_to_replace_for
                moved_farmers +=1
                break
        df_cluster_pickup_outlier = df_clusters_c[(df_clusters_c['cluster_id']==index[0])&(df_clusters_c['pickup']==index[1])]
        overload = np.sum(df_cluster_pickup_outlier[season+'_volume']) - cluster_capacity
        if (overload <= 0):
            success = True
            break
    if not success:
        print("Not possible to make data consistent",file=f)
print("Fixed consistency with moving schedules of %d farmers" % moved_farmers,file=f)

In [172]:
# Parses data into numeric JSON format that can be read by C++
def json_parser(H,N,H_p,N_p,capacities,quantities,mapping, distances, n_trucks, penalties = None):
    N_ = list(range(len(N)))
    H_ = list(range(len(N),len(N)+len(H)))
    capacities_ = [int(capacities[h]*10) for h in H]
    quantities_ = [int(quantities[n]*10) for n in N]
    n_trucks_ = [int(n_trucks[h]) for h in H]
    if penalties:
        penalties_ = []
        for i,h in enumerate(H):
            penalties_.append([])
            for j,n in enumerate(N):
                penalties_[i].append(penalties[h][n])
    file_dict = {"H":H_,
                "N":N_,
                "H_p":H_p.tolist(),
                "N_p":N_p.tolist(),
                "capacities": capacities_,
                "quantities": quantities_,
                "mapping": mapping,
                "distances": distances.tolist(),
                "n_trucks": n_trucks_,
                "penalties": penalties_,
                }
    return file_dict
    

In [135]:
# # Create trucks dataset (Only run once)
# truck_dicts = []
# j = 0
# for i,r in dim_middlemen.iterrows():
#     truck_dic = ast.literal_eval(r['trucks'])
#     for capacity in truck_dic.keys():
#         for i in range(truck_dic[capacity]):
#             truck_dicts.append({
#                     "cluster_id":r['cluster_id'],
#                     "truck_id":"t_"+str(j),
#                     "capacity":int(capacity)
#                 })
#             j += 1
# pd.DataFrame(truck_dicts).to_csv(data_folder+"dim_trucks.csv",index=False)

In [136]:
# Load the trucks dataset
dim_trucks = pd.read_csv(data_folder+'dim_trucks.csv')
# Change names of datasets
df_clusters_original = df_clusters.copy()
df_clusters = df_clusters_c.copy()

In [137]:
# Create a dataset with code to position
positions_dict = {}
for i, row in dim_mills.iterrows():
    positions_dict[row["code"]] = (row['latitude_mill'], row['longitude_mill'])
for i, row in dim_farmers.iterrows():
    positions_dict[row["farmer_id"]+'-'+str(row["plot_number"])] = (row['latitude_farmer'], row['longitude_farmer'])
for i, row in dim_middlemen.iterrows():
    positions_dict[row["cluster_id"]] = (row['latitude_middleman'], row['longitude_middleman'])
for i, row in dim_trucks.iterrows():
    positions_dict[row["truck_id"]] = positions_dict[float(row["cluster_id"])]
unique_id_to_latlon = pd.DataFrame(positions_dict).transpose()
unique_id_to_latlon.columns = ['latitude', 'longitude']
unique_id_to_latlon.to_csv(data_folder+"unique_id_to_latlon.csv")

In [138]:
# Check if farmers sell more than the trucks capacity
inconsistencies = 0
for i, row in df_clusters.iterrows():
    max_capacity = np.max([int(k) for k in row['trucks_dict'].keys()])
    if row[season+'_volume']>max_capacity:
        df_clusters.loc[i,season+'_volume'] = max_capacity
        inconsistencies += 1
print("A total of %d instances where farmer produced more than truck were found and replaced" % inconsistencies,file=f)

In [139]:
# Create dataset of daily production of middlemen
df_clusters.groupby(['cluster_id','pickup'])[['high_volume','low_volume']].sum().to_csv(data_folder+"middlemen_daily_production.csv")

In [140]:
VRPClass = reload(VRPClass)

In [141]:
# Create daily instances
bash_path = data_folder+"instances/bash_daily.sh"
open(bash_path, 'w').close()
for c in clusters:
    for d in range(days_analysis):
        # Extract mill coordinates
        M_p = np.array(dim_mills[dim_mills['code'] == 'SKIP'][["latitude_mill","longitude_mill"]].values)
        trucks = dim_trucks[dim_trucks['cluster_id']==c]['truck_id'].values
        k = len(trucks)
        H = ['h_'+str(i) for i in range(k)]
        H_p = np.array(dim_middlemen[dim_middlemen['cluster_id'] == c][["latitude_middleman","longitude_middleman"]])
        H_p = np.tile(H_p,(k,1))
        capacities = dict(zip(H,dim_trucks[dim_trucks['cluster_id']==c]['capacity'].values))
        mapping_trucks = dict(zip(H,trucks))
        df_cluster_day = df_clusters[(df_clusters['pickup'] == d) & (df_clusters['cluster_id'] == c)]
        n = len(df_cluster_day)
        m = 1
        if (n!=0):
            N = ['n_'+str(i) for i in range(n)]
            M = ['m_'+str(i) for i in range(m)]
            quantities = {f: df_cluster_day[season+"_volume"].values[i] for i,f in enumerate(N)}
            N_p = np.array(df_cluster_day[['latitude_farmer','longitude_farmer']])
            # Create VRP
            vrp = VRPClass.VRP(H, N, H_p, N_p, quantities, capacities, 'road', M, M_p)
            # Dump it in case we want to reuse it
            pickle.dump(vrp, open( results_folder+"vrps/daily/daily_"+"cluster_"+str(c)+"_day_"+str(d)+".p", "wb" ) )

            mapping = {h:(mapping_trucks[h],d, c) for h in H}
            for i,ind in enumerate(df_cluster_day['plot_id']):
                mapping[N[i]] = ind
            n_trucks = dict(zip(H,[1]*len(H)))
            file_dict = json_parser(H,N,H_p,N_p,capacities,quantities,mapping, vrp.distance, n_trucks)
            with open( data_folder+"instances/daily/daily_"+"cluster_"+str(c)+"_day_"+str(d)+".json", "wb" ) as fp:
                json.dump(file_dict, fp)
            file_dict_pickle = {
                "H":H,
                "N":N,
                "M":M,
                "H_p":H_p,
                "N_p":N_p,
                "M_p":M_p,
                "capacities":capacities,
                "quantities":quantities,
                "mapping":mapping,
                "distance":vrp.distance,
                "type_dist":"road",
                "n_trucks":n_trucks,
            }
            pickle.dump( file_dict_pickle, open( data_folder+"instances/daily/daily_"+"cluster_"+str(c)+"_day_"+str(d)+".p", "wb" ) )
            with open(bash_path, 'a') as file:
                file.write("python2 /Users/sergiocamelo/Dropbox/Sergio-Joann/Code/PipelineScripts/load_and_solve_instance.py "+data_folder+"instances/daily/daily_"+"cluster_"+str(c)+"_day_"+str(d) + "\n")

In [90]:
# Create temporal instances
bash_path = data_folder+"instances/bash_temporal.sh"
open(bash_path, 'w').close()

for c in clusters:
    print("Processing config files of cluster: %d" % (c),file=f)
    M_p = np.array(dim_mills[dim_mills['code'] == 'SKIP'][["latitude_mill","longitude_mill"]].values)
    trucks = dim_trucks[dim_trucks['cluster_id']==c]['truck_id'].values
    trucks
    k = len(trucks)
    H = ['h_'+str(i) for i in range(k)]
    H_p = np.array(dim_middlemen[dim_middlemen['cluster_id'] == c][["latitude_middleman","longitude_middleman"]])
    H_p = np.tile(H_p,(k,1))
    H_p
    capacities = dict(zip(H,dim_trucks[dim_trucks['cluster_id']==c]['capacity'].values))
    capacities
    mapping_trucks = dict(zip(H,trucks))
    df_cluster = df_clusters[(df_clusters['cluster_id'] == c)]

    # Construct optimization problem
    n = len(df_cluster)
    m = 1

    N = ['n_'+str(i) for i in range(n)]
    M = ['m_'+str(i) for i in range(m)]


    # Construct the mapping
    mapping_trucks = dict(zip(H,trucks))
    mapping = {h:(mapping_trucks[h],c) for h in H}
    for i,ind in enumerate(df_cluster['plot_id']):
        mapping[N[i]] = ind

    quantities = {f: df_cluster[season_volume].values[i] for i,f in enumerate(N)}
    N_p = np.array(df_cluster[['latitude_farmer','longitude_farmer']])
    vrp = VRPClass.VRP(H, N, H_p, N_p, quantities, capacities, 'road', M, M_p)
    # Dump it in case we want to reuse it
    pickle.dump(vrp, open( results_folder+"vrps/temporal/temporal_"+"cluster_"+str(c)+".p", "wb" ) )

    n_trucks = dict(zip(H,[days_analysis]*len(H)))
    file_dict = json_parser(H,N,H_p,N_p,capacities,quantities,mapping, vrp.distance, n_trucks)
    with open( data_folder+"instances/temporal/temporal_"+"cluster_"+str(c)+".json", "wb" ) as fp:
        json.dump(file_dict, fp)
    file_dict_pickle = {
        "H":H,
        "N":N,
        "M":M,
        "H_p":H_p,
        "N_p":N_p,
        "M_p":M_p,
        "capacities":capacities,
        "quantities":quantities,
        "mapping":mapping,
        "distance":vrp.distance,
        "type_dist":"road",
        "n_trucks":n_trucks
    }
    pickle.dump( file_dict_pickle, open( data_folder+"instances/temporal/temporal_"+"cluster_"+str(c)+".p", "wb" ) )
    with open(bash_path, 'a') as file:
        file.write("python2 /Users/sergiocamelo/Dropbox/Sergio-Joann/Code/PipelineScripts/load_and_solve_instance.py "+data_folder+"instances/temporal/temporal_"+"cluster_"+str(c) + "\n")

In [179]:
# We now solve the spatial problem
# Results spatial
bash_path = data_folder+"instances/bash_spatial.sh"
open(bash_path, 'w').close()

# Extract all trucks that are available
capacities = {}
H_p = np.empty([0,2])
H = []; i = 0
truck_mapping = {}
for c in clusters:
    middle_df_c = dim_middlemen[dim_middlemen.cluster_id == c]
    trucks = dim_trucks[dim_trucks['cluster_id']==c]['truck_id'].values
    capacities_v = dim_trucks[dim_trucks['cluster_id']==c]['capacity'].values
    for j,t in enumerate(trucks):
        H_p = np.vstack([H_p, np.array(middle_df_c[['latitude_middleman','longitude_middleman']])])
        H.append('h_'+str(i))
        truck_mapping['h_'+str(i)] = (t,c)
        capacities['h_'+str(i)] = capacities_v[j]
        i += 1
k = len(capacities)
M_p = np.array(dim_mills[dim_mills['code'] == 'SKIP'][["latitude_mill","longitude_mill"]].values)
for d in range(days_analysis):
    print("Processing config files of dat: %d" % (d),file=f)
    df_day = df_clusters[(df_clusters['pickup'] == d)]
    print(len(df_day))
    n = len(df_day)
    m = 1
    N = ['n_'+str(i) for i in range(n)]
    M = ['m_'+str(i) for i in range(m)]
    mapping = {}
    for h in H:
        mapping[h] = (truck_mapping[h][0],d,truck_mapping[h][1])
    for i,ind in enumerate(df_day['plot_id']):
        mapping[N[i]] = ind
        
    # Get all trucks
    # Create the penalty vector for matching middlemen with farmers that do not correspond to them
    penalties = {}
    for h in H:
        penalties[h] = {}
        for n in N:
            cluster_n = farmer_to_cluster[mapping[n]]
            if truck_mapping[h][1] == cluster_n:
                penalties[h][n] = 0
            else:
                penalties[h][n] = 1
    
    
    # Construct optimization problem
    quantities = {f: df_day[season_volume].values[i] for i,f in enumerate(N)}
    N_p = np.array(df_day[['latitude_farmer','longitude_farmer']])
    vrp = VRPClass.VRP(H, N, H_p, N_p, quantities, capacities, 'road', M, M_p)
    # Dump it in case we want to reuse it
    pickle.dump(vrp, open( results_folder+"vrps/spatial/spatial_"+"day_"+str(d)+".p", "wb" ) )

    n_trucks = dict(zip(H,[1]*len(H)))

    file_dict = json_parser(H,N,H_p,N_p,capacities,quantities,mapping, vrp.distance,n_trucks, penalties)
    print(file_dict)

    with open( data_folder+"instances/spatial/spatial_"+"day_"+str(d)+".json", "wb" ) as fp:
        json.dump(file_dict, fp)

    file_dict_pickle = {
        "H":H,
        "N":N,
        "M":M,
        "H_p":H_p,
        "N_p":N_p,
        "M_p":M_p,
        "capacities":capacities,
        "quantities":quantities,
        "mapping":mapping,
        "distance":vrp.distance,
        "type_dist":"road",
        "n_trucks":n_trucks,
        "penalties":penalties,
    }

    pickle.dump( file_dict_pickle, open( data_folder+"instances/spatial/spatial_"+"day_"+str(d)+".p", "wb" ) )

    with open(bash_path, 'a') as file:
        file.write("python2 /Users/sergiocamelo/Dropbox/Sergio-Joann/Code/PipelineScripts/load_and_solve_instance.py "+data_folder+"instances/spatial/spatial_"+"day_"+str(d) + "\n")


22
{'n_trucks': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'distances': [[0.0, 7224.2048159999995, 4813.002422, 17641.348847, 16062.717133999999, 17866.607167000002, 36327.305649, 2539.276459, 5375.655183, 24173.372781000002, 2686.0620730000005, 11557.663601, 4244.502473, 16575.969839, 15644.664569999999, 15989.47468, 17728.663227, 15793.318994, 15757.511288, 16891.196415000002, 6166.44852, 12967.393199999999, 41984.343246000004, 41984.343246000004, 41984.343246000004, 34077.532799, 32596.53295, 32596.53295, 31287.107983, 34270.136959, 34270.136959, 44443.247503000006, 44443.247503000006, 47525.480888000006, 47525.480888000006, 38759.609577, 38759.609577, 30861.999397000003], [7224.2048159999995, 0.0, 3082.63121, 13532.877634999999, 11954.245922, 13758.135955, 32218.834436999998, 6689.505247, 9571.483971, 20064.901568999998, 6836.290861, 7449.192389, 3184.031261, 12467.498626999999, 11536.193357999999, 11881.003468, 13620.192015, 11684.847781999999, 11649.040076000001, 12782.725

KeyboardInterrupt: 

In [99]:
# Create bash scripts for the cluster

header = """#!/bin/bash
#
#SBATCH --job-name=test
#
#SBATCH --time=%s:00
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=2G

cd ~/vrp_project
source ~/.virtualenvs/vrp_project/bin/activate
module load gurobi/8.0.1_py27
srun python ~/vrp_project/Code/PipelineScripts/load_and_solve_instance.py ~/vrp_project/StandardizedData/instances/%s/%s -p normal,dev,gpu
"""


folder_path = data_folder + 'instances/cluster/'
cluster_path = "~/vrp_project/StandardizedData/instances/cluster/"


submission = folder_path + 'submission_daily.sh'
open(submission, 'w').close()

for c in clusters:
    for d in range(days_analysis):
        instance = "daily_"+"cluster_"+str(c)+"_day_"+str(d)
        path = folder_path + instance + '.sh'
        open(path, 'w').close()
        with open(path, 'a') as file:
            file.write(header % ('00:20',"daily",instance))
        with open(submission, 'a') as file:
            file.write('sbatch ' + cluster_path + instance + '.sh' + '\n')

submission = folder_path + 'submission_temporal.sh'
open(submission, 'w').close()
for c in clusters:
    instance = "temporal_"+"cluster_"+str(c)
    path = folder_path + instance + '.sh'
    open(path, 'w').close()
    with open(path, 'a') as file:
        file.write(header % ('01:30',"temporal",instance))
    with open(submission, 'a') as file:
        file.write('sbatch ' + cluster_path + instance + '.sh' + '\n')
        
submission = folder_path + 'submission_spatial.sh'
open(submission, 'w').close()
for d in range(days_analysis):
    instance = "spatial_"+"day_"+str(d)
    path = folder_path + instance + '.sh'
    open(path, 'w').close()
    with open(path, 'a') as file:
        file.write(header % ('01:30',"spatial",instance))
    with open(submission, 'a') as file:
        file.write('sbatch ' + cluster_path + instance + '.sh' + '\n')

In [98]:
header % (59,"temporal",instance)

'#!/bin/bash\n#\n#SBATCH --job-name=test\n#\n#SBATCH --time=00:59:00\n#SBATCH --ntasks=1\n#SBATCH --cpus-per-task=1\n#SBATCH --mem-per-cpu=2G\n\ncd ~/vrp_project\nsource ~/.virtualenvs/vrp_project/bin/activate\nmodule load gurobi/8.0.1_py27\nsrun python ~/vrp_project/Code/PipelineScripts/load_and_solve_instance.py ~/vrp_project/StandardizedData/instances/temporal/spatial_day_13 -p normal,dev,gpu\n'